# Chapter 4: Geometric Aspects of Linear Algebra

**Before Class**:
* Read Chapter 4 in Savov (2020) and take notes
* Watch the following videos and take notes:
  * [Nonsquare matrices as transformations between dimensions](https://www.3blue1brown.com/lessons/nonsquare-matrices)
  * [Dot products and duality](https://www.3blue1brown.com/lessons/dot-products)
  * [Cross products](https://www.3blue1brown.com/lessons/cross-products)
* Compile a list of questions to bring to class

**During and After Class**:
* Take notes (on paper or a tablet computer)
* Complete this notebook, submit you answer via Gradescope

In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt

## Visualize Linear Systems in 3D

The code below visualizes the rows on a N x 3 matrix in three dimensions.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def visualize_linear_system(A, b, xlim=(-2,2), ylim=(-2,2), return_ax = False):
    """ Visualize rows of linear system as planes in 3D
    
    Arguments:
        A: N x 3 matrix
        B: N x 1 vector
        xlim: tuple of bounds for x dimension
        ylim: tuple of bounds for y dimension
    """

    assert A.shape[1] == 3, "Matrix A must have 3 columns"
    assert A.shape[0] == len(b), "Matrix A and vector b must have the same number of rows"
    
    N = len(b)

    for i in range(N):
        assert np.abs(A[i,2]) > 1E-8, "This function does not support zeros in A[:,2]"

    # Create empty plot
    fig = plt.figure()
    ax = Axes3D(fig, auto_add_to_figure=False)
    fig.add_axes(ax)

    # Create a grid of points for plotting
    X, Y = np.meshgrid(np.linspace(xlim[0],xlim[1],5), np.linspace(ylim[0],ylim[1],5))

    # Loop over equations
    for i in range(N):

        # Define function to calculate z
        func_z = lambda x,y : (b[i] - A[i,0]*x - A[i,1]*y)/A[i,2]
        
        # Calculate values of z over meshgrid
        Z = func_z(X, Y)


        s = "{0:.2f}x + {1:.2f}y + {2:.2f}z = {3:.2f}".format(A[i,0], A[i,1], A[i,2], b[i])
        surf = ax.plot_surface(X, Y, Z, alpha=0.5, label=s)

        # Needed for a legend
        # https://stackoverflow.com/questions/27449109/adding-legend-to-a-surface-plot
        surf._edgecolors2d = surf._edgecolor3d
        surf._facecolors2d = surf._facecolor3d

    # If the linear system is square and full rank
    if A.shape[0] == 3 and np.linalg.matrix_rank(A) == 3:
        x = np.linalg.solve(A,b)
        ax.scatter(x[0], x[1], x[2], marker='x', label="solution", color="k")


    ax.set_xlabel('x',fontsize=18)
    ax.set_ylabel('y',fontsize=18)
    ax.set_zlabel('z',fontsize=18)

    if return_ax:
        return ax
    else:
        plt.legend()
        plt.show()

Let's visualize the equations in the rows of the linear system:

$$
\begin{bmatrix} 1 & 1 & 1 \\ 2 & -1 & 1 \end{bmatrix} {\bf x} = \begin{bmatrix} 0 \\ 2 \end{bmatrix}
$$

In [ ]:
A = np.array([[1,1,1],[2,-1,1]])
b = np.array([0, 2])

visualize_linear_system(A, b)

Geometrically describe the solution of this linear system in a few words.

**Your Answer**:

Let's visualize the equations in the rows of the linear system:

$$
\begin{bmatrix} 1 & 2 & 1 \\ -1 & 0 & -1 \\ 0 & 3 & 1 \end{bmatrix} {\bf x} = \begin{bmatrix} 0 \\ 2 \\ -1 \end{bmatrix}
$$

In [ ]:
### BEGIN SOLUTION
A = np.array([[1, 2, 1],[-1,0,1], [0, 3, 1]])
b = np.array([0, 2, -1])

visualize_linear_system(A, b)
### END SOLUTION

## Projections

### Vector Projected onto a Vector

Let's start by projecting vector ${\bf u}$ onto the line defined by vector ${\bf v}$.

$$ \Pi_{\bf{v}} {\bf u}= \frac{ {\bf u} \cdot {\bf v} }{|| {\bf v} ||^2 } {\bf v}
$$

Now let's perform an example calculation in Python and visualize the result in 3D. We'll reuse some plotting code from [](../03/chapter2.ipynb).

In [ ]:
# define vectors
u = np.array([1,1,1])
v = np.array([2,3,1])

# calculate the l2-norm of v
norm_v = np.linalg.norm(v, ord=2)

proj_u_onto_v = np.dot(u, v)/ norm_v**2 * v

# Create 3D figure and axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Set labels and limits
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_xlim([-1,2])
ax.set_ylim([-1,3])
ax.set_zlim([0,2])

# Plot vectors
def plot_vector(vector, color, label, p = [0,0,0]):
    ''' Plot a 3D vector
        Arguments:
            vector: np array
            color: 'r', 'b', 'g', etc.
            label: string for legend
            p: point to start vector
        
    '''
    assert len(vector) == 3, "Must be a 3D vector"
    assert len(p) == 3, "p must have length 3"

    ax = plt.gca()
    ax.quiver(p[0], p[1], p[2], vector[0], vector[1], vector[2], color=color, arrow_length_ratio=0.1, label=label)

plot_vector(u, 'r', r'${\bf u}$')
plot_vector(v, 'b', r'${\bf v}$')
plot_vector(proj_u_onto_v, 'purple', r'$\Pi_{\bf v} {\bf u}$')

plt.legend(loc='best',ncol=3)
plt.show()

print("v =",v)
print("u =",u)
print("u projected onto v =",proj_u_onto_v)

Verify this calculation with pencil and paper. (It is excellent quiz practice.)

### Vector Projected onto a Plane

Projecting vector $\bf{u}$ onto a plane is a little more involved, requiring three steps:
1. Calculate the vector ${\bf n}$ orthagonal to the plane.
2. Project $\bf{u}$ onto ${\bf n}$.
3. Subtract this projection from $\bf{u}$.

Let's do it in Python and visualize.

In [ ]:
## Step 1: Orthagonal vector for plane
# Consider the plane defined by
A = np.array([[1, 1, 2]])
b = np.array([-1])

# The vector orthagonal to this plane is simply
n = A.copy().flatten()

# And we'll draw our vectors around the point
p = np.array([0, 0, b[0]/A[0,2]])

## Step 2: Projection

# calculate the l2-norm of v
norm_n = np.linalg.norm(n, ord=2)

# calculate the project
proj_u_onto_n = np.dot(u, n)/ norm_n**2 * n

## Step 3: Subtract
proj_u_onto_P = u - proj_u_onto_n

## Step 4: Plot

# Plane
ax = visualize_linear_system(A, b, return_ax=True)

# u
plot_vector(u, 'r', r'${\bf u}$', p=p)

# Orthagonal vector
plot_vector(n, 'b', r'${\bf n}$', p=p)

# Projection (step 2)
plot_vector(proj_u_onto_n, 'purple', r'$\Pi_{\bf n} {\bf u}$', p=p)

# Projection (step 3)
plot_vector(proj_u_onto_P, 'g', r'$\Pi_{P} {\bf u}$', p=p)

plt.legend()
plt.show()

print("u =",u)
print("n =",n)
print("u projected onto n =",proj_u_onto_n)
print("u projected onto P =",proj_u_onto_P)

Verify these calculations with pencil and paper (again, good quiz practice).